###　フォルダーをGDRIVEに作成
GIT CLODEで結構大きなフォルダーなどをいれるため、あとで別ノートでも使えるように緯度だけGITCLONEしたら別のノートでも使えるようにしておきます。

In [ ]:
# ① Google Drive をマウント
from google.colab import drive
drive.mount('/content/drive')
# ② 保存先パス（Drive 内）を設定
code_dir  = "YOURFOLDER"     # ← 好きなフォルダー名をつけてください
code_path = f"/content/drive/MyDrive/{code_dir}"

Mounted at /content/drive


In [ ]:
#保存先フォルダが無ければ作成
import pathlib, os, shutil, subprocess
path = pathlib.Path(code_path)
path.mkdir(parents=True, exist_ok=True)   # 階層をまとめて作成

In [ ]:
!git clone --depth 1 https://github.com/HiDream-ai/MotionPro.git "{code_path}/MotionPro"

Cloning into '/content/drive/MyDrive/プレゼン発表/関東CV_20250824/MotionPro'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 298 (delta 34), reused 275 (delta 29), pack-reused 0 (from 0)
Receiving objects: 100% (298/298), 16.10 MiB | 24.54 MiB/s, done.
Resolving deltas: 100% (34/34), done.
Updating files: 100% (261/261), done.


In [ ]:
# MotionPro フォルダへ移動
%cd "{code_path}/MotionPro"

/content/drive/MyDrive/プレゼン発表/関東CV_20250824/MotionPro


### 不要ライブラリのコメントアウトと必要なライブラリのインストール
requirements.txtにimportlibがあるとエラーになります。

またdeepspeed==0.15.4 も必要です。

最新のdeepspeedだと動かない可能性があります。というか私の環境ではエラーになりました。

In [ ]:
# ① requirements.txt 中の「importlib」をコメントアウト
#    行頭に「# 」を付けます
!sed -i '/^[[:space:]]*importlib/ s/^/# /' requirements.txt

# ② “deepspeed” を末尾に追記（重複しないようにチェック）
!grep -qxF 'deepspeed==0.15.4' requirements.txt || echo 'deepspeed==0.15.4' >> requirements.txt

# 確認
!tail -n+1 requirements.txt

/content/drive/MyDrive/プレゼン発表/関東CV_20250824/MotionPro
tqdm
torch>=2.0.1        # we use 2.4.1
gradio==3.47.1      # NOTE
pytorch-lightning==1.5.0  # NOTE
timm==0.6.7           # NOTE for zero-depth
transformers==4.47.1
xformers==0.0.28 # NOTE: match with torch, this match with torch==2.4.1
av==12.0.0
webdataset==0.2.48
opencv-python
# importlib
einops
scipy
imageio
colorlog
kornia
open_clip_torch
omegaconf
decord
av
termcolor
test-tube
iopath
deepspeed


### モデルを保存する場所を用意
モデルの重みファイルを入れっる場所を用意します。

ちなみにstabilityai/stable-video-diffusion-img2vid
は今回の実験では不要です。
また、入力の画像、マスク画像、軌跡JSONをいれるinput
結果を出力するoutputもここで作成しておきます

In [ ]:
!mkdir -p "/content/drive/MyDrive/YOURFOLDER/MotionPro/models/svd_ckpt"
!mkdir -p "/content/drive/MyDrive/YOURFOLDER/MotionPro/models/motionpro_ckpt"
!mkdir -p "/content/drive/MyDrive/YOURFOLDER/MotionPro/input"
!mkdir -p "/content/drive/MyDrive/YOURFOLDER/MotionPro/output"


In [ ]:
from huggingface_hub import snapshot_download, hf_hub_download

base = f"{code_path}/MotionPro/models"

# ❶ SVD (img2vid) フォルダごと取得
svd_dir = snapshot_download(
    repo_id="stabilityai/stable-video-diffusion-img2vid",
    local_dir=f"{base}/svd_ckpt",            # ← here
    resume_download=True,
    local_dir_use_symlinks=False,
)

# ❷ MotionPro の .pt だけ取得
motion_ckpt1 = hf_hub_download(
    repo_id="HiDream-ai/MotionPro",
    filename="MotionPro-gs_16k.pt",
    local_dir=f"{base}/motionpro_ckpt",      # ← and here
    resume_download=True,
)

motion_ckpt2 = hf_hub_download(
    repo_id="HiDream-ai/MotionPro",
    filename="MotionPro_Dense-gs_14k.pt",
    local_dir=f"{base}/motionpro_ckpt",
    resume_download=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

LICENSE.md: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/6.10G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/196M [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/391M [00:00<?, ?B/s]

MotionPro-gs_16k.pt:   0%|          | 0.00/6.82G [00:00<?, ?B/s]

MotionPro_Dense-gs_14k.pt:   0%|          | 0.00/6.82G [00:00<?, ?B/s]

In [ ]:
!ls -l "/content/drive/MyDrive/YOURFOLDER/MotionPro/models/svd_ckpt"
!ls -l "/content/drive/MyDrive/YOURFOLDER/MotionPro/models/motionpro_ckpt"


total 9353951
-rw------- 1 root root     146654 Jul 23 19:05 comparison.png
drwx------ 2 root root       4096 Jul 23 19:05 feature_extractor
drwx------ 2 root root       4096 Jul 23 19:08 image_encoder
-rw------- 1 root root      11852 Jul 23 19:05 LICENSE.md
-rw------- 1 root root        498 Jul 23 19:05 model_index.json
-rw------- 1 root root   18630497 Jul 23 19:05 output_tile.gif
-rw------- 1 root root       7704 Jul 23 19:05 README.md
drwx------ 2 root root       4096 Jul 23 19:05 scheduler
-rw------- 1 root root 9559625980 Jul 23 19:08 svd.safetensors
drwx------ 2 root root       4096 Jul 23 19:08 unet
drwx------ 2 root root       4096 Jul 23 19:06 vae
total 13319604
-rw------- 1 root root 6819636917 Jul 23 19:13 MotionPro_Dense-gs_14k.pt
-rw------- 1 root root 6819636568 Jul 23 19:12 MotionPro-gs_16k.pt


### トレーニングデータ
COLABではうまく動かせませんでした。
ただし、EC2環境では動かしてるので、一応環境設定の一環としてセルをつくっておきます。

In [ ]:
# トレーニングのデータ
!pip install huggingface_hub

from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="HiDream-ai/MotionPro",
    local_dir="hf_repo_tmp",                       # 保存先ローカルフォルダ
    local_dir_use_symlinks=False,           # シンボリックリンクを使わず実ファイルで取得
    allow_patterns=["data/*"],             # data フォルダ内の全ファイルを対象
)


Fetching 203 files:   0%|          | 0/203 [00:00<?, ?it/s]

MC-Bench.tar:   0%|          | 0.00/795M [00:00<?, ?B/s]

movi_f_cotracker2_patch_4_wind_8.pth:   0%|          | 0.00/204M [00:00<?, ?B/s]

movi_f_raft_patch_4_alpha.pth:   0%|          | 0.00/23.2M [00:00<?, ?B/s]

cotracker_patch_4_wind_8.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

...

visible_mask_14.jpg:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

visible_mask_13.jpg:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

p003_n000.tar:   0%|          | 0.00/449M [00:00<?, ?B/s]

visible_mask_15.jpg:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

'/content/drive/MyDrive/プレゼン発表/関東CV_20250824/MotionPro/hf_repo_tmp'

In [ ]:
!mv hf_repo_tmp/data ./

In [ ]:
!rm -rf hf_repo_tmp